# Set Up

## Importing Libraries

In [1]:
import tensorflow as tf

## Extra

# Data Collection/Preparation

## Collecting Data

## Splitting Data

# Neural Network

## Model Set Up

## Training Model

## Testing Model

## Saving Model Weights